# Results demostration

### Learning for a choosen appliance within each house and avergae learning across all the houses 

The script allows for the aggragation of the learning results from the best alrgorithm found during the conducted study. 

Expected runtime is around 5-15 minutes depending on the PC and the choosen appliance.

In [11]:
from lxml import objectify
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
import datetime as datetime

#### Accuracy score
This parameter determines the % of predictions that are right 

#### Recall score 
This parameter gives the % of predictions that are predicted to be 1 (true, in this case the appliance is on) and it is a correct prediction

In [12]:
#Accuracy and recall scores for all the houses saved in one place
t_data_size = range(1,75)
t = pd.DataFrame(index = t_data_size, columns=['accuracy_1','recall_1','accuracy_2','recall_2',
                                               'accuracy_3','recall_3','accuracy_4','recall_4',
                                               'accuracy_5','recall_5','accuracy_6','recall_6',
                                               'accuracy_7','recall_7','accuracy_8','recall_8',
                                               'accuracy_9','recall_9','accuracy_10','recall_10',
                                               'accuracy_11','recall_11','accuracy_12','recall_12',
                                               'accuracy_13','recall_13','accuracy_14','recall_l4',
                                               'accuracy_15','recall_15','accuracy_16','recall_16',
                                               'accuracy_17','recall_17','accuracy_18','recall_18',
                                               'accuracy_19','recall_19','accuracy_20','recall_20',
                                               'accuracy_21','recall_21'])

In [13]:

def breakdown(a):
    global u
    u = pd.DataFrame(index=app.index, columns=cols) #usage table
    u.set_index(pd.DatetimeIndex(u.index),inplace=True)
    u.fillna(0,inplace=True)
    u['month']=u.index.month
    u['week']=u.index.week
    u['day']=u.index.dayofweek
    u['hour']=u.index.hour
    u['minute']=u.index.minute
    u[a][(app[a]>30)]=1

In [14]:
def RTF(a,ac,r,h):
    
    #### Under Sample data to balance the number of zeros and ones
    
    ones = len(u[a][(u[a] == 1)])
    zero_indices = u[u[a] == 0].index
    random_indices = np.random.choice(zero_indices,ones, replace=False)
    ones = u[u[a] == 1].index
    under_sample_indices = np.concatenate([ones,random_indices])
    under_sample = u.loc[under_sample_indices]
    
    
    #### fit model
    cols = ['month','week','hour','minute','day']
    X = under_sample[cols]
    y = under_sample[a]
    
    if(X.empty):                    # if after undersampling the the traning data becomes empty, the fitting operation will return an error, thus it shouldbe avoided
        print ("Not enough data")
    else:
        # the loop below fits a random forest tree model to the increasing percentage of the data to see the improvement in learning 
        for i in range(1,75): 
            global clf_rf
            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,train_size = i/100, test_size = 1-i/100-0.01)
            clf_rf = RandomForestClassifier(n_estimators=25, random_state=12)
            clf_rf.fit(X_train, y_train)
            y_pred_class = clf_rf.predict(X_test)
            t[ac].loc[i] = clf_rf.score(X_test, y_test)
            t[r].loc[i] = recall_score(y_test, clf_rf.predict(X_test))

        fig, ax = plt.subplots(figsize = (20,7))
        ax.plot(t.index,t[ac])
        ax.plot(t.index,t[r])
        ax.set_title(f'Accuracy of the method {a}, in house {h}')
        ax.set_ylabel('accuracy %')
        ax.set_xlabel('percentage of data used for training')
        ax.legend()
        plt.savefig(f'Figures/{a} in House {h}.png')
    #     plt.show()


In [15]:
# the function defined below creates and average from the 
import math
def average_acc(name,z):
    a = 0
    global m
    m = pd.DataFrame(index=t.index,columns=['mean_accuracy','mean_recall'])
    for column in t:
        if(math.isnan(t[column][1])!=1):
            a = a+1
            
#     t.fillna(0,inplace=True)
    m.mean_accuracy = t[t.columns[t.columns.str.contains('acc')]].mean(axis=1)
    m.mean_recall = t[t.columns[t.columns.str.contains('rec')]].mean(axis=1)
    
    fig, ax = plt.subplots(figsize = (20,7))
    ax.plot(m.index,m.mean_accuracy)
    ax.plot(m.index,m.mean_recall)
    ax.set_title('Mean Accuracy of the method '+str(name)+' in '+str(z)+' houses')
    ax.set_ylabel('accuracy %')
    ax.set_xlabel('percentage of data used for training')
    ax.legend()
#     plt.show()
    plt.savefig(f'Figures/Mean Accuracy of the method {name}.png')

### Choose an appliance

#### Recommended ones that appear in most houses:
TV <br>
Microwave <br>
Kettle <br>
Toaster <br>

In [16]:
a='Kettle'

In [17]:
# c contains the names and order of appliances in each house from the dataset
c = pd.DataFrame(index=range(1,12))

c['1'] = ['aggregate','fridge','freezer','freezer2','Dryer','Washing','Dishwasher','Computer','TV','Eheater','issues']
c['2'] = ['aggregate','fridge','washing_machine','dishwasher','TV','Microwave','Toaster','Hifi','Kettle','Fan','issues']
c['3'] = ['aggragate','Toaster','fridge-freezer','freezer','dryer','dishwasher','washing_machine','TV','Microwave','Kettle','issues']
c['4'] = ['aggragate','fridge','freezer','fridge-freezer','washing_machine','washing_machine2','Desktop_PC','TV','Microwave','Kettle','issues']
c['5'] = ['aggragate','fridge','dryer','washing_machine','dishwasher','Desktop_PC','TV','Microwave','Kettle','Toaster','issues']
c['6'] = ['aggragate','freezer','washing_machine','dishwasher','Desktop_PC','TV','Microwave','Kettle','Toaster','PGM_PC','issues']
c['7'] = ['aggragate','fridge','freezer','freezer2','dryer','washing_machine','dishwasher','TV','Toaster','Kettle','issues']
c['8'] = ['aggragate','fridge','freezer','dryer','washing_machine','Toaster','PC','TV','Microwave','Kettle','issues']
c['9'] = ['aggragate','fridge','dryer','washing_machine','Dishwasher','TV','Microwave','Kettle','Hifi','Electric_Heater','issues']
c['10'] = ['aggragate','blender','Toaster','chest_freezer','fridge','washing_machine','dishwasher','TV','Microwave','K_mix','issues']
c['11'] = ['aggragate','fridge','fridge-freezer','washing_machine','dishwasher','computer','Microwave','Kettle','Router','HiFi','issues']
c['12'] = ['aggragate','fridge','Unk','Unk','computer','Microwave','Kettle','Toaster','TV','Unk','issues']
c['13'] = ['aggragate','TV','freezer','washing_machine','dishwasher','Unk','Network','Microwave','Microwave2','Kettle','issues']
c['15'] = ['aggragate','fridge-freezer','dryer','washingmachine','dishwasher','computer','TV','Microwave','HiFi','Toaster','issues']
c['16'] = ['aggragate','fridge-freezer','fridge_freezer2','electric_heater','electric_heater2','washingmachine','dishwasher','computer','TV','dehumidifier','issues']
c['17'] = ['aggragate','freezer','fridge_freezer','dryer','washingmachine','computer','TV','Microwave','Kettle','Toaster','issues']
c['18'] = ['aggragate','fridge','freezer','fridge_freezer','dryer','washingmachine','dishwasher','computer','TV','Microwave','issues']
c['19'] = ['aggragate','fridge_freezer','washingmachine','TV','Microwave','Kettle','Toaster','Bread-maker','Games-console','Hifi','issues']
c['20'] = ['aggragate','fridge','freezer','dryer','washingmachine','dishwasher','computer','TV','Microwave','Kettle','issues']
c['21'] = ['aggragate','fridge','dryer','washingmachine','dishwasher','foodmixer','TV','Kettle','vivarium','pond_pump','issues']



In [18]:
c

,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21
1,aggregate,aggregate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate,aggragate
2,fridge,fridge,Toaster,fridge,fridge,freezer,fridge,fridge,fridge,blender,fridge,fridge,TV,fridge-freezer,fridge-freezer,freezer,fridge,fridge_freezer,fridge,fridge
3,Freezer,washing_machine,fridge-freezer,freezer,dryer,washing_machine,freezer,freezer,dryer,Toaster,fridge-freezer,Unk,freezer,dryer,fridge_freezer2,fridge_freezer,freezer,washingmachine,freezer,dryer
4,Freezer2,dishwasher,freezer,fridge-freezer,washing_machine,dishwasher,freezer2,dryer,washing_machine,chest_freezer,washing_machine,Unk,washing_machine,washingmachine,electric_heater,dryer,fridge_freezer,TV,dryer,washingmachine
5,Dryer,TV,dryer,washing_machine,dishwasher,Desktop_PC,dryer,washing_machine,Dishwasher,fridge,dishwasher,computer,dishwasher,dishwasher,electric_heater2,washingmachine,dryer,Microwave,washingmachine,dishwasher
6,Washing,Microwave,dishwasher,washing_machine2,Desktop_PC,TV,washing_machine,Toaster,TV,washing_machine,computer,Microwave,Unk,computer,washingmachine,computer,washingmachine,Kettle,dishwasher,foodmixer
7,Dishwasher,Toaster,washing_machine,Desktop_PC,TV,Microwave,dishwasher,PC,Microwave,dishwasher,Microwave,Kettle,Network,TV,dishwasher,TV,dishwasher,Toaster,computer,TV
8,Computer,Hifi,TV,TV,Microwave,Kettle,TV,TV,Kettle,TV,Kettle,Toaster,Microwave,Microwave,computer,Microwave,computer,Bread-maker,TV,Kettle
9,TV,Kettle,Microwave,Microwave,Kettle,Toaster,Toaster,Microwave,Hifi,Microwave,Router,TV,Microwave2,HiFi,TV,Kettle,TV,Games-console,Microwave,vivarium
10,Eheater,Fan,Kettle,Kettle,Toaster,PGM_PC,Kettle,Kettle,Electric_Heater,K_mix,HiFi,Unk,Kettle,Toaster,dehumidifier,Toaster,Microwave,Hifi,Kettle,pond_pump


In [ ]:
z=0 # number of houses that have the given appliance in them 
for i in range(1,22):
    if(i==14): # House 14 is missing in the dataset that is why it is left out
        continue  
    if (c[f'{i}'].str.contains(a).any()==1): # checking if the appliance appers in the house
        path = f'CLEAN_REFIT/CLEAN_House{i}.csv'
        csvapp = pd.read_csv(path,index_col=0,parse_dates=[1])
        csvapp.set_index(pd.DatetimeIndex(csvapp.index),inplace=True)
        app = pd.DataFrame()
        app = csvapp.resample('1T').mean()
        app.columns=c[f'{i}']
        cols=c[f'{i}']
        app = app[app.issues == 0]
        breakdown(a)

        RTF(a,f'accuracy_{i}',f'recall_{i}',i)

        z=z+1
    else:
        print("No", a)

    print(i)

No Kettle
1
2
3
4
5
6
7
8
9
No Kettle
10
11


In [ ]:

average_acc(a,z)
